In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: ignored

In [ ]:
!pip install pytorch-pretrained-bert pytorch-nlp

In [ ]:
!pip install transformers 

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
% matplotlib inline

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

In [ ]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP/data/DataNer.txt", sep=" ", header=None, skip_blank_lines=False, encoding="utf-8")
data.columns = ["word","pos", "upper", "number","tag"]

In [ ]:
print(df.intent.unique())
intents = df.intent.unique()
print("Total categories",len(df.intent.unique()))

In [ ]:
sentences_with_tag = []
sentence_with_tag = []
for w,t in zip(data['word'].values, data['tag'].values):
  if isNaN(w)==False:
    sentence_with_tag.append((w,t))
  if isNaN(w)==True: 
    sentences_with_tag.append(sentence_with_tag)
    sentence_with_tag = []
sentences_with_tag

In [ ]:
sentences = [' '.join([word[0] for word in sent]) for sent in sentences_with_tag]
sentences[2593]

In [ ]:
labels = [[word[1] for word in sent] for sent in sentences_with_tag]
labels[2]
# labels = labels 

In [ ]:
classes = list(filter(lambda x: x not in ["O", np.nan], list(data["tag"].unique())))
classes

In [ ]:
labels_value = classes

In [ ]:
labels_value.append('O')
# intents = labels_value

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[1])

In [ ]:
max_len = 0
# For every sentence...
for sent in sentences:
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

In [ ]:
MAX_LEN = 256

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LEN,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0].shape)
print(attention_masks[0])

In [ ]:
batch_size = 16

data_inputs_rest, validation_inputs, data_labels_rest, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=123, test_size=0.1,shuffle=True)
data_masks_rest, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=123, test_size=0.1,shuffle=True)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
print(validation_data[1])

In [ ]:
from sklearn.model_selection import KFold
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(data_inputs_rest, data_labels_rest))
# kfold = KFold(n_splits=5,shuffle=True,random_state=123)


In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [ ]:
for fold, (train_idx, test_idx) in enumerate(splits):
    print("Training for fold {}".format(fold))
    best_score = 0  

    train_inputs = torch.tensor(data_inputs_rest[train_idx])
    test_inputs = torch.tensor(data_inputs_rest[test_idx])
    train_labels = torch.tensor(data_labels_rest[train_idx])
    test_labels = torch.tensor(data_labels_rest[test_idx])
    batch_size = 16
    train_masks = torch.tensor(data_masks_rest[train_idx])
    test_masks = torch.tensor(data_masks_rest[test_idx])
   
    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

   

    test_data = TensorDataset(test_inputs, test_masks, test_labels)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data , sampler=test_sampler , batch_size=batch_size)  
    model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=45)
    model.cuda()
    
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.0}
    ]
    optimizer = BertAdam(optimizer_grouped_parameters,
                        lr=2e-5,
                        warmup=.1)
    t = [] 
    train_loss_set = []

    # Number of training epochs (authors recommend between 2 and 4)
    epochs = 10

    for _ in trange(epochs, desc="Epoch"):
      
      model.train()
      tr_loss = 0
      nb_tr_examples, nb_tr_steps = 0, 0
    
      for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()

        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        train_loss_set.append(loss.item())    
        loss.backward()
        optimizer.step()

        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

      print("Train loss: {}".format(tr_loss/nb_tr_steps))
        
      model.eval()
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      # Evaluate data for one epoch
      for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

      print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
      model.eval()

      predictions , true_labels = [], []
      for batch in test_dataloader:

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():

          logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
      
        predictions.append(logits)
        true_labels.append(label_ids)
      from sklearn.metrics import matthews_corrcoef
      matthews_set = []

      for i in range(len(true_labels)):
        matthews = matthews_corrcoef(true_labels[i],
                      np.argmax(predictions[i], axis=1).flatten())
        matthews_set.append(matthews)
      from sklearn.metrics import f1_score
      f1_score_set = []
      for i in range(len(true_labels)):
        f1_measure = f1_score(true_labels[i],
                      np.argmax(predictions[i], axis=1).flatten(), average='weighted')
        f1_score_set.append(f1_measure)
      flat_predictions = [item for sublist in predictions for item in sublist]
      flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
      flat_true_labels = [item for sublist in true_labels for item in sublist]
    
    print("f1_score",f1_score(flat_true_labels, flat_predictions,average='weighted'))
    print("matthews_corrcoef",matthews_corrcoef(flat_true_labels, flat_predictions))
    from sklearn.metrics import classification_report
    target_names = intents
    print("Precision, Recall and F1-Score:\n\n",classification_report(flat_true_labels, flat_predictions, target_names=target_names))      
    filename='/content/drive/My Drive/Colab Notebooks/NLP/models/model_bertmultilingual_45_intents.pt'
    torch.save( model,filename)
    print("SAVED SUCESS")
